# annotation

> Pipeline to annotate peptides


In [ ]:
#| default_exp annotation

In [ ]:
#| hide
from nbdev.showdoc import *

In [ ]:
#| export
from typing import Dict, Union
from pyteomics import parser
from pyteomics import mass
from pathlib import Path
from Bio import SeqIO
import pandas as pd
from protein_cutter.core import annotate_peptides_inplace
from protein_cutter.core import fasta_to_peptide_set

Repo root: /Users/mtinti/git_projects/protein_cutter
Test data dir: /Users/mtinti/git_projects/protein_cutter/test_data
Test data exists: True
/Users/mtinti/git_projects/protein_cutter/nbs


In [ ]:
#| export
from pathlib import Path
import os
import yaml
# Get the repository root
if 'GITHUB_WORKSPACE' in os.environ:
    # In GitHub Actions
    REPO_ROOT = Path(os.environ['GITHUB_WORKSPACE'])
else:
    # Local development - find repo root
    REPO_ROOT = Path.cwd()
    while not (REPO_ROOT / 'settings.ini').exists():
        if REPO_ROOT == REPO_ROOT.parent:
            REPO_ROOT = Path.cwd()  # Fallback
            break
        REPO_ROOT = REPO_ROOT.parent

TEST_DATA = REPO_ROOT / 'test_data'
CONFIG_FILES = REPO_ROOT / 'config_files'
print(f"Repo root: {REPO_ROOT}")
print(f"Test data dir: {TEST_DATA}")
print(f"Test data exists: {TEST_DATA.exists()}")

Repo root: /Users/mtinti/git_projects/protein_cutter
Test data dir: /Users/mtinti/git_projects/protein_cutter/test_data
Test data exists: True


In [ ]:
with open(CONFIG_FILES / 'config.yaml', 'r') as stream:
    config = yaml.safe_load(stream)
print(config)   

{'canonical_fasta': 'test_sequence.fa', 'min_pep_length': 5, 'input_file': 'test_spectronaut_pep_out.tsv'}


In [ ]:
#| export
import shutil

#def annotation_pipline():
#    canonical_set = fasta_to_peptide_set(TEST_DATA / config['canonical_fasta'],
#                                         mass_range = (0.0, 4000000.0),
#                                         min_pep_length =config['min_pep_length']
#                                        )
#    print(canonical_set) 
#    canonical_set.update(set(['DASGPAMTEIGEQPWGR','DVAGAVEFWTDR']))
                         
#    annotate_peptides_inplace(TEST_DATA / config['input_file'], canonical_set)

#    spc_out = pd.read_csv(TEST_DATA / config['input_file'],sep='\t')
#    print(spc_out.iloc[:4,-3:].head())
    

#    src = TEST_DATA / "test_spectronaut_pep_out.tsv.bk"
#    dst = TEST_DATA / "test_spectronaut_pep_out.tsv"
    
#    shutil.copy(src, dst)
#    print(f"Restored {dst.name} from backup")    

#annotation_pipline()    

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()